In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#import the csv files
drugsHourly = pd.read_csv(r'C:\Users\ariki\OneDrive\שולחן העבודה\Drugs_Hourly_vaso_demo.csv', encoding = 'ISO-8859-1',)
# drugsHourly = pd.read_csv(r'Drugs_Hourly_vaso_demo.csv', encoding = 'ISO-8859-1',)
deadAlive = pd.read_csv(r'C:\Users\ariki\OneDrive\שולחן העבודה\ver12_proj3_outcomes_3-7_demo.csv', encoding = 'ISO-8859-1',)
# deadAlive = pd.read_csv(r'ver12_proj3_outcomes_3-7_demo.csv', encoding = 'ISO-8859-1',)

In [3]:
ABGLabsHourly = pd.read_excel(r"C:\Users\ariki\Downloads\ABG_Labs_hourly_20000_demo.xlsx", skiprows=[0,1,2], names=["patient_id","hour_counter",
"PaO2-1","PaO2-2","BEvt-1","BEvt-2","PotassiumABG-1","PotassiumABG-2","PH-1","PH-2","IonCalciumABG-1","IonCalciumABG-2",
"LactateABG-1","LactateABG-2","GlucoseABG-1","GlucoseABG-2","Na-ABG-1","Na-ABG-2","Chloride-ABG-1","Chloride-ABG-2",
"PaCO2-1","PaCO2-2"])
vitalHourly = pd.read_excel(r"C:\Users\ariki\Downloads\vital_hourly_20000_demo.xlsx", skiprows=[0,1,2], names=["patient_id","length of stay (hours)",
"hour_counter","ArtBPS","TEMP","HR","ArtBPD","ArtBPM","UR_h"])

In [4]:
ABGLabsHourly.drop(['BEvt-1','BEvt-2','PotassiumABG-1','PotassiumABG-2','PH-1','PH-2','IonCalciumABG-1','IonCalciumABG-2',
'GlucoseABG-1','GlucoseABG-2','Na-ABG-1','Na-ABG-2','Chloride-ABG-1','Chloride-ABG-2','PaCO2-1','PaCO2-2' ], axis=1, inplace=True)
vitalHourly.drop(['ArtBPS','TEMP','HR','ArtBPD','UR_h','length of stay (hours)'],axis=1,inplace=True)

In [5]:
#merge the two table into one
merged_df = pd.merge(drugsHourly, deadAlive, on=['patient_id'], how='inner', indicator=True)
merged_df.drop(['_merge', 'Unnamed: 0'], axis=1, inplace=True)

In [6]:
#remove lines where hour_counter<0
merged_df = merged_df[merged_df.hour_counter>=0]

In [7]:
#remove all patients who didn't get Noradrenaline
allID = merged_df.patient_id.unique()
for i in allID:
    dfID = merged_df[merged_df.patient_id==i]
    noNor = dfID[dfID["Noradrenaline (Norepinephrine)"]==0]
    if(dfID.shape[0]==noNor.shape[0]):
        merged_df = merged_df[merged_df.patient_id!=i]


In [8]:
#merge the two table into one
merged_df1 = pd.merge(merged_df, vitalHourly, on=['patient_id', 'hour_counter'], how='outer', indicator=False)
merged_df1.reset_index(drop=True)
sepsis = pd.merge(merged_df1, ABGLabsHourly, on=['patient_id', 'hour_counter'], how='outer', indicator=True)
sepsis.drop(['_merge'], axis=1, inplace=True)

In [9]:
sepsis.drop(['PaO2-1', 'PaO2-2', 'LactateABG-2'], axis=1, inplace=True)

In [11]:
sepsis.drop(['Vasopressin=Argipressin','Adrenalin (Epinephrine)','Dobutrex (Dobutamine)','Dopamine','ICU_LOS','LOS_Group'],axis=1,inplace=True)

In [12]:
sepsis = sepsis[sepsis['ICU_mortality'].notna()]

In [13]:
sepsis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42026 entries, 0 to 42025
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   patient_id                      42026 non-null  object 
 1   hour_counter                    42026 non-null  int64  
 2   Noradrenaline (Norepinephrine)  42026 non-null  float64
 3   ICU_mortality                   42026 non-null  float64
 4   ArtBPM                          431 non-null    float64
 5   LactateABG-1                    1287 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.2+ MB


In [22]:
sepsis.head()

,patient_id,hour_counter,Noradrenaline (Norepinephrine),ICU_mortality,ArtBPM,LactateABG-1
0,0090EDD0-704B-4AF4-B18D-90A06A5C6514,1,0.40037,0.0,78.00004,5.0
1,0090EDD0-704B-4AF4-B18D-90A06A5C6514,2,0.37020,0.0,80.99998,5.0
2,0090EDD0-704B-4AF4-B18D-90A06A5C6514,3,0.34560,0.0,80.99998,5.0
3,0090EDD0-704B-4AF4-B18D-90A06A5C6514,4,0.29640,0.0,84.00001,5.0
4,0090EDD0-704B-4AF4-B18D-90A06A5C6514,5,0.26196,0.0,84.00001,5.0


In [15]:
sepsis.fillna({'ArtBPM': -1, 'LactateABG-1': -1}, inplace=True)

In [16]:
print(sepsis.size)

252156


In [17]:
import math
last = 0
for i in range (0, int(sepsis.size/6)):  
    if(sepsis.loc[i,'LactateABG-1'] != -1):
        for j in range (last, i + 1):
            a = sepsis.at[i,'LactateABG-1']
            sepsis.iloc[j,5] = a
        last = i + 1



In [18]:
last = 0
for i in range (0, int(sepsis.size/6)):     
    if(sepsis.loc[i,'ArtBPM'] != -1):
        for j in range (last, i + 1):
            a = sepsis.at[i,'ArtBPM']
            sepsis.loc[j,'ArtBPM'] = a
        last = i + 1  

In [19]:
sepsis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42026 entries, 0 to 42025
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   patient_id                      42026 non-null  object 
 1   hour_counter                    42026 non-null  int64  
 2   Noradrenaline (Norepinephrine)  42026 non-null  float64
 3   ICU_mortality                   42026 non-null  float64
 4   ArtBPM                          42026 non-null  float64
 5   LactateABG-1                    42026 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 3.5+ MB


In [20]:
sepsis = sepsis.drop_duplicates(subset=['patient_id','hour_counter'], keep='last')

In [21]:
sepsis.at[11,'ArtBPM']

KeyError: 11

In [ ]:
sepsis.head(30)

In [ ]:
res.head(60)